In [1]:
from io import StringIO

import matplotlib.pyplot as plt
from scipy.io import arff
import seaborn as sns
from loguru import logger
import yaml

from datetime import datetime
import polars as pl
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

# MODEL
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import (
    LabelEncoder, 
    StandardScaler,
    label_binarize
)
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_recall_fscore_support,
    balanced_accuracy_score,
    roc_auc_score,
    roc_curve
)

# Save variables for model
import joblib
import json

import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# PERSONAL FUNCTIONS
from utils import *
from models.main import *
from models.optimizer import ViterbiLiteDecoder
from functions.windows import create_feature_windows # creación de ventanas e ingenieria de características
from functions.build_window_raw import create_raw_windows_250_timesteps_robust
from functions.multimodal import create_multimodal_windows_robust

In [50]:
# dc = {'sit': 96.42857142857143, 'stand': 0.0, 'walk': 98.21958456973294, 'type': 96.15384615384616, 'eat': 100.0}

In [ ]:
dict_preds = {
    'Eat':      {'y': [], 'y_pred': []},
    'Stand':    {'y': [], 'y_pred': []},
    'Walk':     {'y': [], 'y_pred': []},
    'Sit':      {'y': [], 'y_pred': []},
    'Type':     {'y': [], 'y_pred': []}
}

In [ ]:
dic_result = {
            'Eat':      {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []}, 
            'Walk':     {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
            'Sit':      {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
            'Stand':    {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
            'Type':     {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []}
        }

In [62]:
loaded = tf.saved_model.load(r"F:\UPC\Tesis\HARbit-Model\src\cnn_temporal_20_epochs_93\saved_model")
infer = loaded.signatures["serving_default"]
label_encoder = joblib.load(r'F:\UPC\Tesis\HARbit-Model\src\cnn_temporal_20_epochs_93\label_encoder.joblib')


In [63]:
path_base = r"F:\UPC\Tesis\HARbit-Model\src\data\real-data"
archive_date = [archive for archive in os.listdir(r"F:\UPC\Tesis\HARbit-Model\src\data\real-data") if '.json' not in archive]


for data in archive_date:
    path_data = os.path.join(path_base, data)
    file_data = os.listdir(path_data)

    target = data.split('-')[0].title()
    for file in file_data:

        file_path = os.path.join(path_data, file)
        
        with open(file_path, 'rb') as file:
            data = json.load(file)

        gyro_df = data['gyro']
        accel_df = data['accel']

        accel_temp = pl.DataFrame(accel_df)
        gyro_temp = pl.DataFrame(gyro_df)

        
        accel_temp = accel_temp.with_columns(pl.lit('A').alias('Usuario'))
        gyro_temp  = gyro_temp.with_columns(pl.lit('A').alias('Usuario'))

        accel_temp = accel_temp.with_columns(pl.lit(target).alias('gt'))
        gyro_temp = gyro_temp.with_columns(pl.lit(target).alias('gt'))

        df_accel = normalize_columns(accel_temp,
                            user_col_name  = "Usuario", 
                            timestamp_col_name = "timestamp", 
                            label_col_name = "gt", 
                            x_col_name = "x", 
                            y_col_name = "y", 
                            z_col_name = "z"
                        )

        df_gyro = normalize_columns(gyro_temp, 
                            user_col_name  = "Usuario", 
                            timestamp_col_name = "timestamp", 
                            label_col_name = "gt", 
                            x_col_name = "x", 
                            y_col_name = "y", 
                            z_col_name = "z"
                        )
        
        df_accel = convert_timestamp(df_accel)
        df_gyro = convert_timestamp(df_gyro)

        X_all, y_all, subjects_all, metadata_all = create_raw_windows_250_timesteps_robust(
            df=df_accel,
            window_seconds=5,
            overlap_percent=50,
            sampling_rate=20,
            target_timesteps=100,
            min_data_threshold=0.8,  # 80% mínimo de datos
            max_gap_seconds=1.0      # Máximo 1 segundo de gap
        )

        X_tensor = tf.constant(X_all, dtype=tf.float32)
        y_pred = infer(X_tensor)
        y_all = label_encoder.transform(y_all)

        # Ejemplo: y_pred con probabilidades
        y_pred_probs = list(y_pred.values())[0].numpy()

        # Convertir a clases predichas (índice del máximo)
        y_pred_classes = np.argmax(y_pred_probs, axis=1)


        # Comparar con y_real (asegúrate de que y_real tenga etiquetas 0..N-1)
        acc = accuracy_score(y_all, y_pred_classes)
        
        precision, recall, f1, _ = precision_recall_fscore_support(
                y_all, y_pred_classes, average='macro', zero_division=0
            )
        
        dic_result[target]['Accuracy'].append(acc*100)
        dic_result[target]['F1-score'].append(f1*100)
        dic_result[target]['Precision'].append(precision*100)
        dic_result[target]['Recall'].append(recall*100)

        dict_preds[target]['y'].extend(y_all)
        dict_preds[target]['y_pred'].extend(y_pred_classes)        

🔧 Configuración de ventanas RAW ROBUSTA:
  Duración: 5s
  Timesteps objetivo: 100
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Duración de ventana: 5s
  Paso entre ventanas: 2.50s
👤 Usuario A, Actividad Eat: 6531 muestras
   Duración total: 150.3s
  ✅ Creadas 59 ventanas válidas

📊 RESUMEN DE VALIDACIÓN:
  Ventanas intentadas: 59
  Ventanas creadas: 59
  Tasa de éxito: 100.0%

📊 RESULTADO FINAL (ROBUSTO):
  Forma de X: (59, 100, 3)
  Forma de y: (59,)
  Total ventanas: 59
  Usuarios únicos: 1
  Actividades únicas: ['Eat']
🔧 Configuración de ventanas RAW ROBUSTA:
  Duración: 5s
  Timesteps objetivo: 100
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Duración de ventana: 5s
  Paso entre ventanas: 2.50s
👤 Usuario A, Actividad Eat: 6488 muestras
   Duración total: 150.4s
  ✅ Creadas 59 ventanas válidas

📊 RESUMEN DE VALIDACIÓN:
  Ventanas intentadas: 59
  

In [64]:
for activity in dict_preds.keys():

    y_true = dict_preds[activity]['y']
    y_pred = dict_preds[activity]['y_pred']
    
    acc = accuracy_score(y_true, y_pred)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average='macro', zero_division=0
        )
    print(f"{activity}")
    print(f"Accuracy: {acc*100}")
    print(f"F1-score: {f1*100}")
    print(f"Precision: {precision*100}")
    print(f"Recall: {recall*100}")
    print("-" * 50)

Eat
Accuracy: 75.87064676616916
F1-score: 12.325722368155184
Precision: 14.285714285714285
Recall: 10.83866382373845
--------------------------------------------------
Stand
Accuracy: 0.0
F1-score: 0.0
Precision: 0.0
Recall: 0.0
--------------------------------------------------
Walk
Accuracy: 94.35146443514645
F1-score: 19.418729817007534
Precision: 20.0
Recall: 18.87029288702929
--------------------------------------------------
Sit
Accuracy: 89.44954128440367
F1-score: 47.21549636803874
Precision: 50.0
Recall: 44.72477064220183
--------------------------------------------------
Type
Accuracy: 97.65886287625418
F1-score: 49.407783417935704
Precision: 50.0
Recall: 48.82943143812709
--------------------------------------------------


In [65]:
from sklearn.metrics import classification_report

y_true_global = []
y_pred_global = []

for activity in dict_preds.keys():
    y_true_global.extend(dict_preds[activity]['y'])
    y_pred_global.extend(dict_preds[activity]['y_pred'])

labels = np.unique(y_true_global)  # clases presentes en los datos
target_names = label_encoder.inverse_transform(labels)

print(classification_report(
    y_true_global, y_pred_global,
    labels=labels,
    target_names=target_names
))

              precision    recall  f1-score   support

         Eat       0.56      0.76      0.64       402
         Sit       0.73      0.89      0.81       218
       Stand       0.00      0.00      0.00       205
        Type       0.93      0.98      0.95       299
        Walk       1.00      0.94      0.97       956

   micro avg       0.83      0.81      0.82      2080
   macro avg       0.64      0.71      0.67      2080
weighted avg       0.78      0.81      0.79      2080



In [67]:
accuracy_score(y_true_global, y_pred_global) * 100

81.4423076923077

In [2]:
# Cargar datos
path_base = r"F:\UPC\Tesis\HARbit-Model\src\data\wisdm-dataset\raw\watch"
sensor_data = load_sensors_separately(path_base)
df_gyroscope = sensor_data['gyro']
# df_gyro = df_gyro.rename({'X': 'X_gyro', 'Y': 'Y_gyro', 'Z': 'Z_gyro'})

df_accelerometer = sensor_data['accel']
# df_accel = df_accel.rename({'X': 'X_accel', 'Y': 'Y_accel', 'Z': 'Z_accel'})

print(f"Giroscopio: {len(df_gyro)} muestras")
print(f"Acelerómetro: {len(df_accel)} muestras")

2025-09-30 00:20:05.966 | INFO     | utils.data_loading:load_sensors_separately:64 - Cargando datos de gyro...
2025-09-30 00:20:13.655 | INFO     | utils.data_loading:load_sensors_separately:64 - Cargando datos de accel...


NameError: name 'df_gyro' is not defined

In [154]:
X_acc, y_acc, subjects_acc, metadata_acc = create_raw_windows_250_timesteps_robust(
            df=df_accelerometer,
            window_seconds=5,
            overlap_percent=50,
            sampling_rate=20,
            target_timesteps=100,
            min_data_threshold=0.8,  # 80% mínimo de datos
            max_gap_seconds=1.0      # Máximo 1 segundo de gap
        )

🔧 Configuración de ventanas RAW ROBUSTA:
  Duración: 5s
  Timesteps objetivo: 100
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Duración de ventana: 5s
  Paso entre ventanas: 2.50s
👤 Usuario 1600.0, Actividad A: 3604 muestras
   Duración total: 179.8s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad B: 3605 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad C: 3605 muestras
   Duración total: 179.8s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad D: 3606 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad E: 3605 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad F: 3605 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad G: 3604 muestras
   Duración total: 179.8s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad H:

In [155]:
X_gyro, y_gyro, subjects_gyro, metadata_gyro = create_raw_windows_250_timesteps_robust(
            df=df_gyroscope,
            window_seconds=5,
            overlap_percent=50,
            sampling_rate=20,
            target_timesteps=100,
            min_data_threshold=0.8,  # 80% mínimo de datos
            max_gap_seconds=1.0      # Máximo 1 segundo de gap
        )

🔧 Configuración de ventanas RAW ROBUSTA:
  Duración: 5s
  Timesteps objetivo: 100
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Duración de ventana: 5s
  Paso entre ventanas: 2.50s
👤 Usuario 1600.0, Actividad A: 3602 muestras
   Duración total: 179.7s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad B: 3604 muestras
   Duración total: 179.8s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad C: 3604 muestras
   Duración total: 179.8s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad D: 3604 muestras
   Duración total: 179.8s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad E: 3604 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad F: 3604 muestras
   Duración total: 179.9s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad G: 3603 muestras
   Duración total: 179.8s
  ✅ Creadas 70 ventanas válidas
👤 Usuario 1600.0, Actividad H:

In [ ]:
X_all, y_all, subjects_all, metadata_all = create_multimodal_windows_robust(
            df_accel = df_accelerometer, 
            df_gyro = df_gyroscope,
            window_seconds=5,
            overlap_percent=50,
            sampling_rate=20,
            target_timesteps=100,
            min_data_threshold=0.8,  # 80% mínimo de datos
            max_gap_seconds=1.0      # Máximo 1 segundo de gap
        )

🔧 CONFIGURACIÓN MULTIMODAL (Accel + Gyro):
  Duración: 5s
  Timesteps objetivo: 100
  Canales totales: 6
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Tolerancia sincronización: 50ms
  ACCEL preparado: 3777045 muestras
  GYRO preparado: 3440341 muestras

🔄 SINCRONIZANDO SENSORES...
  📊 Datos originales:
    Acelerómetro: 3,777,045 muestras
    Giroscopio: 3,440,341 muestras
  ⏰ Rango temporal común: 2722977.0s
  📊 Datos sincronizados:
    Acelerómetro: 3,777,043 muestras
    Giroscopio: 3,440,341 muestras

📏 PARÁMETROS TEMPORALES:
  Duración de ventana: 5s
  Paso entre ventanas: 2.50s

👤 Usuario 1600.0, A:
   Accel: 3604 muestras
   Gyro: 3602 muestras
   ✅ Creadas 70 ventanas válidas

👤 Usuario 1600.0, B:
   Accel: 3605 muestras
   Gyro: 3604 muestras
   ✅ Creadas 70 ventanas válidas

👤 Usuario 1600.0, C:
   Accel: 3605 muestras
   Gyro: 3604 muestras
   ✅ Creadas 70 ventanas válidas

👤 Usuario 1600.0, D:
   Accel: 36

In [151]:
def synchronize_sensor_data(df_accel, df_gyro, sampling_rate=20):
    """
    Sincroniza datos de acelerómetro y giroscopio:
    1. Encuentra rango temporal común.
    2. Filtra ambos sensores a ese rango.
    3. Reindexa e interpola a un timeline uniforme con la misma longitud.
    
    Args:
        df_accel, df_gyro: DataFrames con columnas ['Timestamp','X','Y','Z']
        sampling_rate: Frecuencia objetivo en Hz (default=20Hz → cada 50ms)
    
    Returns:
        df_accel_sync, df_gyro_sync: DataFrames sincronizados e interpolados
    """
    print(f"\n🔄 SINCRONIZANDO SENSORES...")

    # Asegurar que los timestamps son datetime
    df_accel['Timestamp'] = pd.to_datetime(df_accel['Timestamp'])
    df_gyro['Timestamp']  = pd.to_datetime(df_gyro['Timestamp'])

    # Rango temporal común
    common_start = max(df_accel['Timestamp'].min(), df_gyro['Timestamp'].min())
    common_end   = min(df_accel['Timestamp'].max(), df_gyro['Timestamp'].max())

    print(f"  Rango temporal común: {common_start} → {common_end} "
          f"({(common_end - common_start).total_seconds():.2f}s)")

    # Filtrar al rango común
    df_accel = df_accel[(df_accel['Timestamp'] >= common_start) & (df_accel['Timestamp'] <= common_end)]
    df_gyro  = df_gyro[(df_gyro['Timestamp']  >= common_start) & (df_gyro['Timestamp']  <= common_end)]

    # Crear timeline uniforme
    freq_ms = int(1000 / sampling_rate)  # periodo en ms
    timeline = pd.date_range(start=common_start, end=common_end, freq=f"{freq_ms}ms")

    print("¿Índice accel es único?", df_accel['Timestamp'].is_unique)
    print("¿Índice gyro es único?", df_gyro['Timestamp'].is_unique)
    print("Timestamps accel ejemplo:", df_accel['Timestamp'].head(10).tolist())
    print("Timestamps timeline ejemplo:", timeline[:10].tolist())

    # Reindexar e interpolar acelerómetro
    df_accel_sync = (
        df_accel.set_index('Timestamp')
                .reindex(timeline)
                .interpolate(method='linear')
                .reset_index()
                .rename(columns={'index':'Timestamp'})
    )

    # Reindexar e interpolar giroscopio
    df_gyro_sync = (
        df_gyro.set_index('Timestamp')
               .reindex(timeline)
               .interpolate(method='linear')
               .reset_index()
               .rename(columns={'index':'Timestamp'})
    )

    print(f"  Accel sincronizado: {len(df_accel_sync)} muestras")
    print(f"  Gyro sincronizado:  {len(df_gyro_sync)} muestras")
    print(f"  Timeline: {timeline[0]} → {timeline[-1]} ({len(timeline)} pasos)")

    return df_accel_sync, df_gyro_sync


def analyze_sync_quality(df_accel, df_gyro, tolerance_ms):
    """Analiza la calidad de sincronización entre sensores"""
    
    # Convertir a arrays de tiempo
    accel_times = df_accel['Timestamp'].values.astype('int64') 
    gyro_times = df_gyro['Timestamp'].values.astype('int64') 
    
    # Encontrar coincidencias aproximadas
    matched_pairs = 0
    time_differences = []
    
    for accel_time in accel_times[:min(1000, len(accel_times))]:  # Muestrear para eficiencia
        time_diffs = np.abs(gyro_times - accel_time)
        min_diff_ns = np.min(time_diffs)
        min_diff_ms = min_diff_ns / 1e6
        
        if min_diff_ms <= tolerance_ms:
            matched_pairs += 1
            time_differences.append(min_diff_ms)
    
    if time_differences:
        avg_diff = np.mean(time_differences)
        max_diff = np.max(time_differences)
        print(f"  Calidad sincronización:")
        print(f"    Pares coincidentes: {matched_pairs}/{min(1000, len(accel_times))}")
        print(f"    Diferencia promedio: {avg_diff:.1f}ms")
        print(f"    Diferencia máxima: {max_diff:.1f}ms")
    else:
        print(f"  ⚠️ Baja sincronización: pocos pares dentro de {tolerance_ms}ms")


In [83]:
def synchronize_sensor_data_optimized(df_accel, df_gyro, sync_tolerance_ms):
    """
    Versión optimizada de sincronización con análisis eficiente de memoria
    """
    print(f"\n🔄 SINCRONIZANDO SENSORES (OPTIMIZADO)...")
    
    # Convertir timestamps a nanosegundos para precisión
    accel_times_ns = df_accel['Timestamp'].astype('int64')
    gyro_times_ns = df_gyro['Timestamp'].astype('int64')
    
    print(f"  📊 Datos originales:")
    print(f"    Acelerómetro: {len(df_accel):,} muestras")
    print(f"    Giroscopio: {len(df_gyro):,} muestras")
    
    # Encontrar rango temporal común
    common_start = max(accel_times_ns.min(), gyro_times_ns.min())
    common_end = min(accel_times_ns.max(), gyro_times_ns.max())
    
    print(f"  ⏰ Rango temporal común: {(common_end - common_start) / 1e9:.1f}s")
    
    # Filtrar datos al rango común
    accel_mask = (accel_times_ns >= common_start) & (accel_times_ns <= common_end)
    gyro_mask = (gyro_times_ns >= common_start) & (gyro_times_ns <= common_end)
    
    df_accel_sync = df_accel[accel_mask].copy()
    df_gyro_sync = df_gyro[gyro_mask].copy()
    
    print(f"  📊 Datos sincronizados:")
    print(f"    Acelerómetro: {len(df_accel_sync):,} muestras")
    print(f"    Giroscopio: {len(df_gyro_sync):,} muestras")
    
    # Análisis de calidad optimizado
    analyze_sync_quality_optimized(df_accel_sync, df_gyro_sync, sync_tolerance_ms)
    
    return df_accel_sync, df_gyro_sync

def analyze_sync_quality_optimized(df_accel, df_gyro, tolerance_ms, chunk_size=100):
    """
    Analiza la calidad de sincronización usando vectorización por chunks para evitar MemoryError
    """
    print(f"  🔍 Analizando sincronización con chunks de {chunk_size:,} muestras...")
    
    # Convertir a enteros en nanosegundos
    accel_times = df_accel['Timestamp'].values.astype('int64')  # ns
    gyro_times = df_gyro['Timestamp'].values.astype('int64')    # ns
    
    tolerance_ns = tolerance_ms * 1e6  # Convertir a nanosegundos
    
    total_accel = len(accel_times)
    matched_pairs = 0
    all_valid_diffs = []
    
    # Procesar en chunks para evitar MemoryError
    for start_idx in range(0, 1000, chunk_size):
        end_idx = min(start_idx + chunk_size, total_accel)
        accel_chunk = accel_times[start_idx:end_idx]
        
        # Calcular diferencias solo para este chunk
        diffs_ns = np.abs(accel_chunk[:, None] - gyro_times[None, :])  # shape = (chunk_size, N_gyro)
        
        # Para cada muestra del chunk, encontrar la diferencia mínima
        min_diffs_ns = np.min(diffs_ns, axis=1)
        min_diffs_ms = min_diffs_ns / 1e6
        
        # Filtrar las que cumplen con el umbral
        valid_mask = min_diffs_ms <= tolerance_ms
        chunk_matches = np.sum(valid_mask)
        matched_pairs += chunk_matches
        
        # Guardar diferencias válidas para estadísticas
        if chunk_matches > 0:
            valid_diffs_chunk = min_diffs_ms[valid_mask]
            all_valid_diffs.extend(valid_diffs_chunk)
        
        # Progreso cada 100k muestras
        if (end_idx % 10000) == 0 or end_idx == total_accel:
            progress = (end_idx / total_accel) * 100
            print(f"    Progreso: {progress:.1f}% ({end_idx:,}/{total_accel:,})")
    
    # Mostrar resultados
    if matched_pairs > 0:
        avg_diff = float(np.mean(all_valid_diffs))
        max_diff = float(np.max(all_valid_diffs))
        sync_rate = (matched_pairs / total_accel) * 100
        
        print(f"  ✅ Calidad sincronización (análisis completo):")
        print(f"    Pares coincidentes: {matched_pairs:,}/{total_accel:,} ({sync_rate:.1f}%)")
        print(f"    Diferencia promedio: {avg_diff:.1f} ms")
        print(f"    Diferencia máxima: {max_diff:.1f} ms")
        print(f"    Diferencia mínima: {float(np.min(all_valid_diffs)):.1f} ms")
    else:
        print(f"  ⚠️ Baja sincronización: ningún par dentro de {tolerance_ms} ms")